## DNN Models for Energy Forecasting

The code to use for DNN model of the energy forecast project

In [1]:
import tensorflow as tf
print(tf.__version__)

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


In [11]:
price = tf.placeholder(tf.float32, shape=(100,50,1))
weather = tf.placeholder(tf.float32, shape=(100,50,1))

tf.concat([price, weather], axis=-1)

<tf.Tensor 'concat:0' shape=(100, 50, 2) dtype=float32>

In [12]:
price = tf.placeholder(tf.float32, shape=(100, 50))
# weather = tf.placeholder(tf.float32, shape=(100,50,1))

# tf.concat([price, weather], axis=-1)
price

<tf.Tensor 'Placeholder_5:0' shape=(100, 50) dtype=float32>

In [2]:
# change these to try this notebook out
BUCKET = 'atos-asl'
PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1'
REGION = 'us-central1'

In [3]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'  

In [4]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [ ]:
%%bash
OUTDIR=/tmp/trained_data/cnn/
JOBNAME=energy_forecasting_cnn_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
rm -rf $OUTDIR
gcloud ml-engine local train \
  --module-name=atos-asl.cnn_task \
  --package-path=$(pwd)/atos-asl \
  --job-dir=$OUTDIR \
  -- \
  --eval_set=gs://${BUCKET}/datasets/timeserie_price_weekly_eval.csv \
  --train_set=gs://${BUCKET}/datasets/timeserie_price_weekly_train.csv \
  --output_dir=${OUTDIR}

In [31]:
%%bash
OUTDIR=gs://${BUCKET}/trained_data/cnn/daily/
JOBNAME=energy_forecasting_cnn_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --runtime-version=$TFVERSION \
  --scale-tier=BASIC_GPU \
  --staging-bucket=gs://$BUCKET \
  --module-name=atos-asl.cnn_task \
  --package-path=$(pwd)/atos-asl \
  --job-dir=$OUTDIR \
  -- \
  --eval_set=gs://${BUCKET}/datasets/timeserie_price_daily_eval_fix.csv \
  --train_set=gs://${BUCKET}/datasets/timeserie_price_daily_train_fix.csv \
  --output_dir=${OUTDIR} \
  --learning_rate 0.1 \
  --training_steps 10000

gs://atos-asl/trained_data/cnn/daily/ us-central1 energy_forecasting_cnn_180917_232618
jobId: energy_forecasting_cnn_180917_232618
state: QUEUED


Removing gs://atos-asl/trained_data/cnn/daily/#1537226278912393...
Removing gs://atos-asl/trained_data/cnn/daily/events.out.tfevents.1537226279.cmle-training-6479928648897215838#1537226279742564...
Removing gs://atos-asl/trained_data/cnn/daily/graph.pbtxt#1537226281989887...
/ [3/3 objects] 100% Done                                                       
Operation completed over 3 objects.                                              
Job [energy_forecasting_cnn_180917_232618] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe energy_forecasting_cnn_180917_232618

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs energy_forecasting_cnn_180917_232618


In [35]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/trained_data/'.format(BUCKET))

TensorBoard was started successfully with pid 1007. Click here to access it.

1007

In [34]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 32128
